# <center> Add appropriate Emoji at the end of each sentence using LSTM </center>

## <center> Deep RNN Many to One </center>

In [1]:
import numpy as np
import emoji

from emo_utils import *
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
np.random.seed(1)

Using TensorFlow backend.


## Load Train/Test data

In [2]:
X_train_raw, Y_train_raw = read_csv('data/train_emoji.csv')
X_test_raw, Y_test_raw = read_csv('data/test_emoji.csv')

print(X_train_raw.shape,X_train_raw[1])
print(Y_train_raw.shape,label_to_emoji(Y_train_raw[1]))

(132,) I am proud of your achievements
(132,) 😄


## Lets use Pre-trained Glove Word Embeddings

### Embedding Matrix

- It should be matrix of `(vocab_len, vec_size)` , i.e (50,400K) for our word to vec
- The column location should be identified by using `word_to_index[]` array

In [3]:
def read_embedding_matrix():
    """
    return pretrained embedding matrix (vocab_len,vec_size)
    Use (400K,50) glove word vectors to load the matrix.
    """
    word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    vec_len = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    emb_matrix = np.zeros((vocab_len,vec_len))
    
    # Set each row "index" of the embedding matrix to be the word vector representation 
    # of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    return (emb_matrix,word_to_index)

In [4]:
emb_matrix,word_to_index = read_embedding_matrix()
vocab_len, vec_len = emb_matrix.shape
print('matrix=',emb_matrix.shape,'word->indexmapper.len =',len(word_to_index))

matrix= (400001, 50) word->indexmapper.len = 400000


# Feature Engineering

### Transform Label ( To OneHot Encoding)

In [5]:
Y_train = to_categorical(Y_train_raw, num_classes = 5)
Y_test = to_categorical(Y_test_raw, num_classes = 5)
print('Y_train_raw.shape =', Y_train_raw.shape, 'Y_train.shape =', Y_train.shape)

Y_train_raw.shape = (132,) Y_train.shape = (132, 5)


### Transform Features (Words to indices which will be converted later into Vec in embedding Layer)

In [6]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.    

    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- {'word':index} =>index starts from 1
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j+1
                
    return X_indices

## Lets pad all words into a common max length to help vectorization/batch processing.

In [7]:
def find_max_length_of_words(x):
    return len(max(x,key=lambda x:len(x.split())).split())

In [8]:
maxLen = max(find_max_length_of_words(X_train_raw),find_max_length_of_words(X_test_raw))
X_train = sentences_to_indices(X_train_raw,word_to_index,maxLen)
X_test = sentences_to_indices(X_test_raw,word_to_index,maxLen)

print('maxLen = ',maxLen)
print('X_train_raw.shape = ', X_train_raw.shape,'\n', 'X_train_raw[0] =', X_train_raw[0])
print('X_train.shape =',X_train.shape,'\n','X_train[0] = ',X_train[0])

maxLen =  10
X_train_raw.shape =  (132,) 
 X_train_raw[0] = never talk to me again
X_train.shape = (132, 10) 
 X_train[0] =  [259914. 352214. 360915. 239105.  47887.      0.      0.      0.      0.
      0.]


## Create Keras Embedding Layer with pretrained weights

In [9]:
def pretrained_embedding_layer(emb_matrix,vocab_len,vec_len,trainable):
    # Define Keras embedding layer with the correct output/input sizes, make it trainable. 
    embedding_layer = Embedding(vocab_len,vec_len,trainable=trainable)

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer


### LSTM

LSTM takes input shape as (m,Tx,Nx).
- `m` number of examples in a given batch
- `Tx` number of sequences in each example. Convert all examples into same length for easy vector processing. However, the inference can be done on any size.
- `Nx` - Vector size.

Now when you do single call LSTM(input_matrix), it will do one full cycle of forward pass for all words in the sequence. You have option to get final output(A) of Tx th word, or get out put of all passes by using the variable return_sequences. 

You have to get the output from LSTM(A) and use a DenseLayer and softmax activation to convert it into output Y-hat.

## Model Creation

INPUT -> EMBEDDING -> LSTM -> DROPOUT -> LSTM ->DROPOUT -> DENSE -> SOFTMAX (OUTPUT) -> LOSS (FOR OPTIMIZER)

In [10]:
def Emojify_Model(input_shape):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,000 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Create Input Tensor for the Model
    sentence_indices = X = Input(shape=input_shape, dtype='int32',name='main_input')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    X = pretrained_embedding_layer(emb_matrix,vocab_len,vec_len,trainable=False)(X)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Lets take output from all sequences, so that we can use another LSTM.
    X = LSTM(128,return_sequences='True')(X)
    
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    # Lets take only the last output so that we can get the results.
    X = LSTM(128)(X)
    
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    
    # Propagate X through a Dense layer.
    X = Dense(5)(X)

    #  with softmax activation to get back a batch of 5-dimensional vectors.
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=[sentence_indices], outputs=[X])
    
    ### END CODE HERE ###
    
    return model

In [11]:
model = Emojify_Model((maxLen,))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


W1120 14:26:45.604747 4370101696 deprecation_wrapper.py:119] From /Users/madhukandasamy/miniconda3/envs/py3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1120 14:26:45.620751 4370101696 deprecation_wrapper.py:119] From /Users/madhukandasamy/miniconda3/envs/py3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1120 14:26:45.621896 4370101696 deprecation_wrapper.py:119] From /Users/madhukandasamy/miniconda3/envs/py3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1120 14:26:45.631613 4370101696 deprecation_wrapper.py:119] From /Users/madhukandasamy/miniconda3/envs/py3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_defau

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
__________

In [14]:
model.fit(X_train, Y_train, epochs = 1000, batch_size = 32, shuffle=True)

Epoch 1/1000
132/132 [==============================] - 0s 969us/step - loss: 0.2232 - acc: 0.8864
Epoch 2/1000
132/132 [==============================] - 0s 834us/step - loss: 0.2111 - acc: 0.8864
Epoch 3/1000
132/132 [==============================] - 0s 797us/step - loss: 0.1792 - acc: 0.9394
Epoch 4/1000
132/132 [==============================] - 0s 741us/step - loss: 0.1813 - acc: 0.9470
Epoch 5/1000
132/132 [==============================] - 0s 758us/step - loss: 0.1635 - acc: 0.9242
Epoch 6/1000
132/132 [==============================] - 0s 824us/step - loss: 0.1462 - acc: 0.9318
Epoch 7/1000
132/132 [==============================] - 0s 784us/step - loss: 0.1494 - acc: 0.9242
Epoch 8/1000
132/132 [==============================] - 0s 791us/step - loss: 0.1213 - acc: 0.9545
Epoch 9/1000
132/132 [==============================] - 0s 797us/step - loss: 0.1380 - acc: 0.9318
Epoch 10/1000
132/132 [==============================] - 0s 780us/step - loss: 0.1230 - acc: 0.9621
Epoch 11/

132/132 [==============================] - 0s 707us/step - loss: 0.0029 - acc: 1.0000
Epoch 84/1000
132/132 [==============================] - 0s 725us/step - loss: 0.0021 - acc: 1.0000
Epoch 85/1000
132/132 [==============================] - 0s 752us/step - loss: 0.0027 - acc: 1.0000
Epoch 86/1000
132/132 [==============================] - 0s 685us/step - loss: 0.0018 - acc: 1.0000
Epoch 87/1000
132/132 [==============================] - 0s 719us/step - loss: 0.0025 - acc: 1.0000
Epoch 88/1000
132/132 [==============================] - 0s 704us/step - loss: 0.0020 - acc: 1.0000
Epoch 89/1000
132/132 [==============================] - 0s 660us/step - loss: 0.0020 - acc: 1.0000
Epoch 90/1000
132/132 [==============================] - 0s 674us/step - loss: 0.0023 - acc: 1.0000
Epoch 91/1000
132/132 [==============================] - 0s 690us/step - loss: 0.0020 - acc: 1.0000
Epoch 92/1000
132/132 [==============================] - 0s 665us/step - loss: 0.0024 - acc: 1.0000
Epoch 93/1000


132/132 [==============================] - 0s 719us/step - loss: 7.1455e-04 - acc: 1.0000
Epoch 164/1000
132/132 [==============================] - 0s 723us/step - loss: 7.1010e-04 - acc: 1.0000
Epoch 165/1000
132/132 [==============================] - 0s 711us/step - loss: 4.6094e-04 - acc: 1.0000
Epoch 166/1000
132/132 [==============================] - 0s 743us/step - loss: 5.8465e-04 - acc: 1.0000
Epoch 167/1000
132/132 [==============================] - 0s 714us/step - loss: 9.3046e-04 - acc: 1.0000
Epoch 168/1000
132/132 [==============================] - 0s 731us/step - loss: 6.4104e-04 - acc: 1.0000
Epoch 169/1000
132/132 [==============================] - 0s 677us/step - loss: 4.1742e-04 - acc: 1.0000
Epoch 170/1000
132/132 [==============================] - 0s 688us/step - loss: 6.2096e-04 - acc: 1.0000
Epoch 171/1000
132/132 [==============================] - 0s 686us/step - loss: 6.8784e-04 - acc: 1.0000
Epoch 172/1000
132/132 [==============================] - 0s 734us/ste

132/132 [==============================] - 0s 724us/step - loss: 4.0216e-04 - acc: 1.0000
Epoch 242/1000
132/132 [==============================] - 0s 678us/step - loss: 2.9098e-04 - acc: 1.0000
Epoch 243/1000
132/132 [==============================] - 0s 681us/step - loss: 2.4527e-04 - acc: 1.0000
Epoch 244/1000
132/132 [==============================] - 0s 676us/step - loss: 4.9501e-04 - acc: 1.0000
Epoch 245/1000
132/132 [==============================] - 0s 679us/step - loss: 3.7452e-04 - acc: 1.0000
Epoch 246/1000
132/132 [==============================] - 0s 674us/step - loss: 4.3241e-04 - acc: 1.0000
Epoch 247/1000
132/132 [==============================] - 0s 665us/step - loss: 2.8308e-04 - acc: 1.0000
Epoch 248/1000
132/132 [==============================] - 0s 664us/step - loss: 3.4498e-04 - acc: 1.0000
Epoch 249/1000
132/132 [==============================] - 0s 650us/step - loss: 3.2027e-04 - acc: 1.0000
Epoch 250/1000
132/132 [==============================] - 0s 672us/ste

132/132 [==============================] - 0s 842us/step - loss: 1.5890e-04 - acc: 1.0000
Epoch 319/1000
132/132 [==============================] - 0s 918us/step - loss: 2.3532e-04 - acc: 1.0000
Epoch 320/1000
132/132 [==============================] - 0s 867us/step - loss: 2.3445e-04 - acc: 1.0000
Epoch 321/1000
132/132 [==============================] - 0s 681us/step - loss: 2.3352e-04 - acc: 1.0000
Epoch 322/1000
132/132 [==============================] - 0s 835us/step - loss: 3.2164e-04 - acc: 1.0000
Epoch 323/1000
132/132 [==============================] - 0s 744us/step - loss: 2.1356e-04 - acc: 1.0000
Epoch 324/1000
132/132 [==============================] - 0s 732us/step - loss: 1.8661e-04 - acc: 1.0000
Epoch 325/1000
132/132 [==============================] - 0s 726us/step - loss: 3.0712e-04 - acc: 1.0000
Epoch 326/1000
132/132 [==============================] - 0s 724us/step - loss: 3.7467e-04 - acc: 1.0000
Epoch 327/1000
132/132 [==============================] - 0s 708us/ste

...

132/132 [==============================] - 0s 738us/step - loss: 1.5093e-04 - acc: 1.0000
Epoch 629/1000
132/132 [==============================] - 0s 706us/step - loss: 9.5017e-05 - acc: 1.0000
Epoch 630/1000
132/132 [==============================] - 0s 681us/step - loss: 3.7717e-05 - acc: 1.0000
Epoch 631/1000
132/132 [==============================] - 0s 661us/step - loss: 7.9045e-05 - acc: 1.0000
Epoch 632/1000
132/132 [==============================] - 0s 674us/step - loss: 9.0560e-05 - acc: 1.0000
Epoch 633/1000
132/132 [==============================] - 0s 658us/step - loss: 5.5053e-05 - acc: 1.0000
Epoch 634/1000
132/132 [==============================] - 0s 652us/step - loss: 7.6305e-05 - acc: 1.0000
Epoch 635/1000
132/132 [==============================] - 0s 623us/step - loss: 4.7901e-05 - acc: 1.0000
Epoch 636/1000
132/132 [==============================] - 0s 641us/step - loss: 7.8457e-05 - acc: 1.0000
Epoch 637/1000
132/132 [==============================] - 0s 645us/ste

132/132 [==============================] - 0s 702us/step - loss: 6.9664e-05 - acc: 1.0000
Epoch 707/1000
132/132 [==============================] - 0s 711us/step - loss: 5.5818e-05 - acc: 1.0000
Epoch 708/1000
132/132 [==============================] - 0s 685us/step - loss: 5.4930e-05 - acc: 1.0000
Epoch 709/1000
132/132 [==============================] - 0s 1ms/step - loss: 4.9418e-05 - acc: 1.0000
Epoch 710/1000
132/132 [==============================] - 0s 1ms/step - loss: 6.8140e-05 - acc: 1.0000
Epoch 711/1000
132/132 [==============================] - 0s 1ms/step - loss: 3.4564e-05 - acc: 1.0000
Epoch 712/1000
132/132 [==============================] - 0s 738us/step - loss: 1.0561e-04 - acc: 1.0000
Epoch 713/1000
132/132 [==============================] - 0s 715us/step - loss: 4.1205e-05 - acc: 1.0000
Epoch 714/1000
132/132 [==============================] - 0s 707us/step - loss: 5.8530e-05 - acc: 1.0000
Epoch 715/1000
132/132 [==============================] - 0s 686us/step - lo

132/132 [==============================] - 0s 739us/step - loss: 3.3415e-05 - acc: 1.0000
Epoch 785/1000
132/132 [==============================] - 0s 724us/step - loss: 3.3417e-05 - acc: 1.0000
Epoch 786/1000
132/132 [==============================] - 0s 713us/step - loss: 4.9905e-05 - acc: 1.0000
Epoch 787/1000
132/132 [==============================] - 0s 722us/step - loss: 5.2187e-05 - acc: 1.0000
Epoch 788/1000
132/132 [==============================] - 0s 756us/step - loss: 3.8692e-05 - acc: 1.0000
Epoch 789/1000
132/132 [==============================] - 0s 704us/step - loss: 3.1114e-05 - acc: 1.0000
Epoch 790/1000
132/132 [==============================] - 0s 712us/step - loss: 4.3850e-05 - acc: 1.0000
Epoch 791/1000
132/132 [==============================] - 0s 696us/step - loss: 3.9782e-05 - acc: 1.0000
Epoch 792/1000
132/132 [==============================] - 0s 713us/step - loss: 6.6480e-05 - acc: 1.0000
Epoch 793/1000
132/132 [==============================] - 0s 732us/ste

132/132 [==============================] - 0s 821us/step - loss: 3.9955e-05 - acc: 1.0000
Epoch 863/1000
132/132 [==============================] - 0s 727us/step - loss: 2.6982e-05 - acc: 1.0000
Epoch 864/1000
132/132 [==============================] - 0s 736us/step - loss: 4.9604e-05 - acc: 1.0000
Epoch 865/1000
132/132 [==============================] - 0s 695us/step - loss: 5.5244e-05 - acc: 1.0000
Epoch 866/1000
132/132 [==============================] - 0s 700us/step - loss: 3.1319e-05 - acc: 1.0000
Epoch 867/1000
132/132 [==============================] - 0s 701us/step - loss: 4.3517e-05 - acc: 1.0000
Epoch 868/1000
132/132 [==============================] - 0s 691us/step - loss: 3.5068e-05 - acc: 1.0000
Epoch 869/1000
132/132 [==============================] - 0s 754us/step - loss: 4.0798e-05 - acc: 1.0000
Epoch 870/1000
132/132 [==============================] - 0s 798us/step - loss: 4.5302e-05 - acc: 1.0000
Epoch 871/1000
132/132 [==============================] - 0s 688us/ste

132/132 [==============================] - 0s 741us/step - loss: 3.0770e-05 - acc: 1.0000
Epoch 941/1000
132/132 [==============================] - 0s 726us/step - loss: 3.0082e-05 - acc: 1.0000
Epoch 942/1000
132/132 [==============================] - 0s 682us/step - loss: 2.8082e-05 - acc: 1.0000
Epoch 943/1000
132/132 [==============================] - 0s 708us/step - loss: 1.9901e-05 - acc: 1.0000
Epoch 944/1000
132/132 [==============================] - 0s 721us/step - loss: 3.8792e-05 - acc: 1.0000
Epoch 945/1000
132/132 [==============================] - 0s 704us/step - loss: 1.9127e-05 - acc: 1.0000
Epoch 946/1000
132/132 [==============================] - 0s 825us/step - loss: 3.3176e-05 - acc: 1.0000
Epoch 947/1000
132/132 [==============================] - 0s 727us/step - loss: 3.3105e-05 - acc: 1.0000
Epoch 948/1000
132/132 [==============================] - 0s 703us/step - loss: 1.8438e-05 - acc: 1.0000
Epoch 949/1000
132/132 [==============================] - 0s 719us/ste

## Lets test our model

In [15]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['I am not happy','I am happy','I like cricket','I hate you','cute'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
for i in range(len(x_test)):
    print(x_test[i] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices[[i]]))))

I am not happy 😞
I am happy 😄
I like cricket 😄
I hate you 😞
cute ❤️
